# Phenome-Wide analysis on TOPMed studies

# Environment set-up

### System requirements
- Python 3.6 or later
- pip & bash interpreter

### Installation of external dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install -r requirements.txt

In [ ]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

import PicSureHpdsLib
import PicSureClient

from python_lib.utils import get_multiIndex_variablesDict, get_dic_renaming_vars, match_dummies_to_varNames, joining_variablesDict_onCol
from python_lib.HPDS_connection_manager import tokenManager

In [ ]:
print("NB: This Jupyter Notebook has been written using PIC-SURE API following versions:\n- PicSureHpdsLib: 1.1.0\n- PicSureClient: 0.1.0")
print("The PIC-SURE API libraries versions you've been downloading are:\n- PicSureHpdsLib: {0}\n- PicSureClient: {1}".format(PicSureHpdsLib.__version__, PicSureClient.__version__))

In [ ]:
# Pandas DataFrame display options
pd.set_option("max.rows", 435)

# Matplotlib display parameters
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 14
fig_size[1] = 8
plt.rcParams["figure.figsize"] = fig_size
font = {'weight' : 'bold',
        'size'   : 12}
plt.rc('font', **font)

## Connecting to a PIC-SURE network

In [ ]:
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file = "token.txt"

In [ ]:
token = tokenManager(token_file).get_token()

In [ ]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, token, allowSelfSignedSSL=True)
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource(resource_id)

### 1. Retrieving variables dictionary from HPDS Database

In [ ]:
plain_variablesDict = resource.dictionary().find().DataFrame()

In [ ]:
plain_variablesDict.iloc[[1,2], :]

In [ ]:
variablesDict = get_multiIndex_variablesDict(plain_variablesDict)

## Names of different studies

In [ ]:
studies_names = variablesDict.index.get_level_values(0).unique()

In [ ]:
pprint(studies_names)

In [ ]:
mask_harmonized_variables = variablesDict.loc["DCC Harmonized data set", "simplified_varName"].str.match("^age.*")
harmonized_variables = variablesDict.loc["DCC Harmonized data set", "varName"].loc[~mask_harmonized_variables]
harmonized_variables_names = harmonized_variables.to_list()

## Name of harmonized studies

- NHLBI TOPMed: Genetics of Cardiometabolic Health in the Amish Atherosclerosis Risk in Communities (ARIC) Cohort
- Barbados Genetics of Asthma Study
- CARDIA Cohort
- Cleveland Clinic Atrial Fibrillation Study
- NHLBI Cleveland Family Study (CFS) Candidate Gene Association Resource (CARe) Cardiovascular Health Study (CHS) Cohort
- Genetic Epidemiology of COPD (COPDGene)
- NHLBI TOPMed: The Genetic Epidemiology of Asthma in Costa Rica
- Framingham Cohort
- Genetic Epidemiology Network of Arteriopathy (GENOA)
- Genetics of Lipid Lowering Drugs and Diet Network (GOLDN) Lipidomics Study Hispanic Community Health Study /Study of Latinos (HCHS/SOL)
- Heart and Vascular Health Study (HVH)
- Jackson Heart Study (JHS) Cohort
- NHGRI Genome-Wide Association Study of Venous Thromboembolism (GWAS of VTE) Multi-Ethnic Study of Atherosclerosis (MESA) Cohort
- Massachusetts General Hospital trial Fibrillation Study
- Partners HealthCare Biobank
- Study of African Americans, Asthma, Genes and Environment Study
- Genome-wide Association Study of Adiposity in Samoans
- The Vanderbilt AF Ablation Registry
- The Vanderbilt Atrial Fibrillation Registry
- Women’s Genome Health Study
- Women’s Health Initiative

# Multiple PheWAS

In [ ]:
mask_harmonized = study_info_df["harmonized"] == True
mask_nb_var = study_info_df["number var"] < 2000
study_names = study_info_df.loc[mask_harmonized & mask_nb_var,:].index.tolist()

In [ ]:
study_names

In [ ]:
from python_lib.PheWAS_funcs import PheWAS

dependent_var_name = variablesDict\
           .loc[variablesDict.index.get_level_values(2) == 'Indicates whether subject ever regularly smoked cigarettes.', "varName"]\
           .values[0]
dic_pvalues = {}
for study_name in study_names:
    print(study_name)
    study_info_df = pd.read_csv("studies_info.csv", index_col=0)
    subject_id_var = study_info_df.loc[study_name, "ID varName"]
    dic_pvalues[study_name] = PheWAS(study_name,
       dependent_var_name, 
      study_info_df, 
      variablesDict, 
                    resource)

In [ ]:
list_study_names = [] 
list_frames = []
for study_name, study_pvalues in dic_pvalues.items():
    list_study_names.append(study_name)
    list_frames.append(pd.DataFrame.from_dict(study_pvalues, orient='index'))
df_pvalues = pd.concat(list_frames, keys=list_study_names)
df_pvalues = df_pvalues.rename_axis(index=["study", "varName"])\
.reset_index("varName", drop=False)\
.rename({0: "pvalues"}, axis=1)



### Distribution of p-values (univariate tests)

In [ ]:
pd.Series([v for v in df_pvalues["pvalues"].values]).plot.hist(bins=30)
plt.suptitle("Distribution of individual p-values",
             weight="bold",
            fontsize=15)

### 5. Multiple hypotheses testing correction: Bonferroni Method

In order to handle the multiple testing problem (increase in the probability to "discover" false statistical associations), we will use the Bonferroni correction method. Although many other multiple comparisons exist, Bonferroni is the most straightforward to use, because it doesn't require assumptions about variables correlation. Other PheWAS analysis also use False Discovery Rate controlling procedures ([see](https://en.wikipedia.org/wiki/False_discovery_rate)).

In a nutshell, Bonferonni allows to calculate a corrected "statistical significant threshold" according to the number of test performed. Every p-value below this threshold will be deemed statistically significant.

In [ ]:
adjusted_alpha = 0.05/len(df_pvalues["pvalues"])

In [ ]:
%%capture
df_pvalues["p_adj"] = df_pvalues["pvalues"] / len(df_pvalues["pvalues"])
df_pvalues['log_p'] = -np.log10(df_pvalues['pvalues'])
df_pvalues = df_pvalues.sort_index()
df_pvalues["group"] = df_pvalues.index
variablesDict = joining_variablesDict_onCol(variablesDict,
                                              df_pvalues,
                                              left_col="varName",
                                              right_col="varName")

In [ ]:
print("Bonferonni adjusted significance threshold: {0:.2E}".format(adjusted_alpha))

## 6. Result visualisations: Manhattan plot

Manhattan plot is the classical results representation of a PheWAS analysis. It plots every each tested phenotypical variables on the X-axis, against its *-log(pvalue)* on the Y-axis. The horizontal line represent the adjusted significance level threshold.

In [ ]:
mask = variablesDict["pvalues"].isna()
df_results = variablesDict.loc[~mask,:].copy().replace([np.inf, -np.inf], np.nan)
df_results = df_results.loc[~df_results["log_p"].isna().values,:]

In [ ]:
df_results["simplified_varName"] = df_results["simplified_varName"].str.replace("[0-9]+[A-z]*", "").to_frame()
order_studies = df_results.index.get_level_values(0).unique().tolist()[::-1]
df_results = df_results.reindex(order_studies, level=0)
fig = plt.figure()
ax = fig.add_subplot(111)
colors = plt.get_cmap('Set1')
x_labels = []
x_labels_pos = []

y_lims = (0, df_results["log_p"].max(skipna=True) + 50)
threshold_top_values = df_results["log_p"].sort_values(ascending=False)[0:6][-1]

df_results["ind"] = np.arange(1, len(df_results)+1)
#df_results["group"] = df_results["group"].str.replace("[0-9]", "")
df_grouped = df_results.groupby(('group'))
for num, (name, group) in enumerate(df_grouped):
    group.plot(kind='scatter', x='ind', y='log_p',color=colors.colors[num % len(colors.colors)], ax=ax, s=20)
    x_labels.append(name)
    x_labels_pos.append((group['ind'].iloc[-1] - (group['ind'].iloc[-1] - group['ind'].iloc[0])/2)) # Set label in the middle

    pair_ind = 0 # To shift label which might overlap because to close
    for n, row in group.iterrows():
#        if pair_ind %2 == 0:
#            shift = 1.1
#        else:
#            shift = -1.1
        if row["log_p"] > threshold_top_values:
            ax.text(row['ind'] + 3, row["log_p"] + 0.05, row["simplified_varName"], rotation=0, alpha=1, size=8, color="black")
#            pair_ind += 1
                
ax.set_xticks(x_labels_pos)
ax.set_xticklabels(x_labels)
ax.set_xlim([0, len(df_results) +1])
ax.set_ylim(y_lims)
ax.set_ylabel('-log(p-values)', style="italic")
ax.set_xlabel('Phenotypes', fontsize=15)
ax.axhline(y=-np.log10(adjusted_alpha), linestyle=":", color="black", label="Bonferonni Adjusted Threshold")
plt.xticks(fontsize = 9,rotation=90)
plt.yticks(fontsize = 8)
plt.title("Statistical Association Between Exposition Status and Phenotypes", 
          loc="center",
          style="oblique", 
          fontsize = 20,
         y=1)
xticks = ax.xaxis.get_major_ticks()
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles = handles, labels = labels, loc = "upper left")
plt.show()

Overall, it appears that most of the tested phenotypes covariates are above the adjusted threshold of significant association. However, it is not surprising at all, given the nature of our dependent variable: a lot of those variables are by nature tied directly to the COPD status.

This code can be used directly with any other variable present in the variable Dictionary. It only need to change the `dependent_var_name` value.

# Odds Ratio table

In [ ]:
- Univarai

In [ ]:

query = query.select().add(variables)